조강현

In [ ]:
client = speech.SpeechClient()

with open('test4.wav', 'rb') as fp:
    audio_f = fp.read()

audio = speech.RecognitionAudio(content=audio_f)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code="ko-KR",
)

# Detects speech in the audio file
response = client.recognize(config=config, audio=audio)

for result in response.results:
    print("Transcript: {}".format(result.alternatives[0].transcript))

NameError: ignored

박석원

In [ ]:
from __future__ import division

import re
import sys

from google.cloud import speech
from google.cloud.speech import enums
from google.cloud.speech import types
import pyaudio
from six.moves import queue
from threading import Thread
import time

# Audio recording parameters
RATE = 16000
CHUNK = int(RATE / 10)  # 100ms

class MicrophoneStream(object):
    """Opens a recording stream as a generator yielding the audio chunks."""
    def __init__(self, rate, chunk):
        self._rate = rate
        self._chunk = chunk

        # Create a thread-safe buffer of audio data
        self._buff = queue.Queue()
        self.closed = True
        self.isPause = False

    def __enter__(self):
        self._audio_interface = pyaudio.PyAudio()
        self._audio_stream = self._audio_interface.open(
            format=pyaudio.paInt16,
            channels=1, rate=self._rate,
            input=True, frames_per_buffer=self._chunk,
            stream_callback=self._fill_buffer,
        )

        self.closed = False

        return self

    def __exit__(self, type, value, traceback):
        self._audio_stream.stop_stream()
        self._audio_stream.close()


        self.closed = True
        self._buff.put(None)
        self._audio_interface.terminate()

    def pause(self):
        if self.isPause == False:
            self.isPause = True


    def resume(self):
        if self.isPause == True:
            self.isPause = False


    def status(self):
        return self.isPause

    def _fill_buffer(self, in_data, frame_count, time_info, status_flags):
        """Continuously collect data from the audio stream, into the buffer."""
        if self.isPause == False:
            self._buff.put(in_data)
        #else
        return None, pyaudio.paContinue

    def generator(self):
        while not self.closed:
            chunk = self._buff.get()
            if chunk is None:
                return

            data = [chunk]

            while True:
                try:
                    chunk = self._buff.get(block=False)
                    if chunk is None:
                        return
                    data.append(chunk)
                except queue.Empty:
                    break

            yield b''.join(data)


# [END audio_stream]

In [ ]:
class Gspeech(Thread):
    def __init__(self):
        Thread.__init__(self)

        self.language_code = 'ko-KR'  # a BCP-47 language tag

        self._buff = queue.Queue()

        self.client = speech.SpeechClient()
        self.config = types.RecognitionConfig(
            encoding=enums.RecognitionConfig.AudioEncoding.LINEAR16,
            sample_rate_hertz=RATE,
            language_code=self.language_code)
        self.streaming_config = types.StreamingRecognitionConfig(
            config=self.config,
            interim_results=True)

        self.mic = None
        self.status = True

        self.daemon = True
        self.start()

    def __eixt__(self):
        self._buff.put(None)

    def run(self):
        with MicrophoneStream(RATE, CHUNK) as stream:
            self.mic = stream
            audio_generator = stream.generator()
            requests = (types.StreamingRecognizeRequest(audio_content=content)
                        for content in audio_generator)

            responses = self.client.streaming_recognize(self.streaming_config, requests)

            # Now, put the transcription responses to use.
            self.listen_print_loop(responses, stream)
        self._buff.put(None)
        self.status = False

    def pauseMic(self):
        if self.mic is not None:
            self.mic.pause()

    def resumeMic(self):
        if self.mic is not None:
            self.mic.resume()

    # 인식된 Text 가져가기
    def getText(self, block = True):
        return self._buff.get(block=block)

    # 음성인식 처리 루틴
    def listen_print_loop(self, responses, mic):
        num_chars_printed = 0
        try:
            for response in responses:
                if not response.results:
                    continue

                result = response.results[0]
                if not result.alternatives:
                    continue

                transcript = result.alternatives[0].transcript
                overwrite_chars = ' ' * (num_chars_printed - len(transcript))
                if not result.is_final:
                    sys.stdout.write(transcript + overwrite_chars + '\r')
                    sys.stdout.flush()
                    #### 추가 ### 화면에 인식 되는 동안 표시되는 부분.
                    num_chars_printed = len(transcript)
                else:
                    # 큐에 넣는다.
                    self._buff.put(transcript+overwrite_chars)
                    num_chars_printed = 0
        except:
            return

In [ ]:
import gspeech
import time

def main():
    gsp = gspeech.Gspeech()
    while True:
        # 음성 인식 될때까지 대기 한다.
        stt = gsp.getText()
        if stt is None:
            break
        print(stt)
        time.sleep(0.01)
        #끝내자는 명령이 들어오면 프로그램 종료
        if ('끝내자' in stt):
            break


if __name__ == '__main__':
    main()

양보승

In [ ]:
import requests
import json
kakao_speech_url = "https://kakaoi-newtone-openapi.kakao.com/v1/recognize"

rest_api_key = '9d0eae41906de495bcb46d11839db95f'

headers = {
    "Content-Type": "application/octet-stream",
    "X-DSS-Service": "DICTATION",
    "Authorization": "KakaoAK " + rest_api_key,
}

with open('010.wav', 'rb') as fp:
    audio = fp.read()

res = requests.post(kakao_speech_url, headers=headers, data=audio)

result_json_string = res.text[res.text.index('{"type":"finalResult"'):res.text.rindex('}') + 1]
result=json.loads(result_json_string)
print(result)
print(result['value'])

{'type': 'finalResult', 'value': '아아 흠', 'nBest': [{'value': '아아 흠', 'score': 17}, {'value': '아아 지금', 'score': 0}, {'value': '아아 흠흠', 'score': 0}, {'value': '아아 음', 'score': 0}, {'value': '아 예 지금', 'score': 0}, {'value': '아 왜 지금', 'score': 0}, {'value': '아 나 지금', 'score': 0}, {'value': '아 그 지금', 'score': 0}], 'voiceProfile': {'registered': False, 'authenticated': False}, 'durationMS': 17280, 'qmarkScore': 0, 'gender': 0}
아아 흠


강성일

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving ornate-magnet-270605-fd7a3a4c42d3.json to ornate-magnet-270605-fd7a3a4c42d3.json


In [ ]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/content/ornate-magnet-270605-fd7a3a4c42d3.json"

In [ ]:
!echo $GOOGLE_APPLICATION_CREDENTIALS

/content/ornate-magnet-270605-fd7a3a4c42d3.json


In [ ]:
!pip install --upgrade google-cloud-speech

Requirement already up-to-date: google-cloud-speech in /usr/local/lib/python3.7/dist-packages (2.4.0)


In [ ]:
!pip install google-cloud-texttospeech

     |████████████████████████████████| 71kB 1.9MB/s 


In [ ]:
!sudo apt-get install -q -y timidity libsndfile1

E: dpkg was interrupted, you must manually run 'sudo dpkg --configure -a' to correct the problem. 


In [ ]:
!pip install pydub numba==0.48 librosa music21

     |████████████████████████████████| 3.6MB 2.8MB/s 
     |████████████████████████████████| 20.2MB 1.7MB/s 
  Found existing installation: llvmlite 0.34.0
    Uninstalling llvmlite-0.34.0:
      Successfully uninstalled llvmlite-0.34.0
  Found existing installation: numba 0.51.2
    Uninstalling numba-0.51.2:
      Successfully uninstalled numba-0.51.2


In [ ]:
import tensorflow as tf
import tensorflow_hub as hub
from google.cloud import speech
from google.cloud import texttospeech
import numpy as np
import matplotlib.pyplot as plt
import librosa
from librosa import display as librosadisplay
import sys
import requests
import os
import logging
import math
import statistics
import sys
import soundfile 

from IPython.display import Audio, Javascript
from scipy.io import wavfile

from base64 import b64decode

import music21
from pydub import AudioSegment

logger = logging.getLogger()
logger.setLevel(logging.ERROR)

print("tensorflow: %s" % tf.__version__)
#print("librosa: %s" % librosa.__version__)

tensorflow: 2.5.0


In [ ]:
RECORD = """
const sleep  = time => new Promise(resolve => setTimeout(resolve, time))
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader()
  reader.onloadend = e => resolve(e.srcElement.result)
  reader.readAsDataURL(blob)
})
var record = time => new Promise(async resolve => {
  stream = await navigator.mediaDevices.getUserMedia({ audio: true })
  recorder = new MediaRecorder(stream)
  chunks = []
  recorder.ondataavailable = e => chunks.push(e.data)
  recorder.start()
  await sleep(time)
  recorder.onstop = async ()=>{
    blob = new Blob(chunks)
    text = await b2text(blob)
    resolve(text)
  }
  recorder.stop()
})
"""

def record(sec=5):
  try:
    from google.colab import output
  except ImportError:
    print('No possible to import output from google.colab')
    return ''
  else:
    print('Recording')
    display(Javascript(RECORD))
    s = output.eval_js('record(%d)' % (sec*1000))
    fname = 'heykakao.wav'
    print('Saving to', fname)
    b = b64decode(s.split(',')[1])
    with open(fname, 'wb') as f:
      f.write(b)
    return fname

In [ ]:
uploaded_file_name = record(10)

Recording


<IPython.core.display.Javascript object>

Saving to heykakao.wav


In [ ]:
EXPECTED_SAMPLE_RATE = 16000

def convert_audio_for_model(user_file, output_file='converted_audio_file.wav'):
  audio = AudioSegment.from_file(user_file)
  audio = audio.set_frame_rate(EXPECTED_SAMPLE_RATE).set_channels(1)
  audio.export(output_file, format="wav")
  return output_file

In [ ]:
converted_audio_file = convert_audio_for_model(uploaded_file_name)
data, samplerate = soundfile.read(converted_audio_file)
soundfile.write(converted_audio_file, data, samplerate, subtype='PCM_16')

In [ ]:
# Loading audio samples from the wav file:
sample_rate, audio_samples = wavfile.read(converted_audio_file, 'rb')
print(audio_samples.)
# Show some basic information about the audio.
duration = len(audio_samples)/sample_rate
print(f'Sample rate: {sample_rate} Hz')
print(f'Total duration: {duration:.2f}s')
print(f'Size of the input: {len(audio_samples)}')

# Let's listen to the wav file.
Audio(audio_samples, rate=sample_rate)

SyntaxError: ignored

In [ ]:
client = speech.SpeechClient()

with open('converted_audio_file.wav', 'rb') as fp:
    audio_f = fp.read()

audio = speech.RecognitionAudio(content=audio_f)

config = speech.RecognitionConfig(
    encoding=speech.RecognitionConfig.AudioEncoding.LINEAR16,
    sample_rate_hertz=16000,
    language_code="ko-KR",
)

# Detects speech in the audio file
response = client.recognize(config=config, audio=audio)

for result in response.results:
    print("Transcript: {}".format(result.alternatives[0].transcript))

Transcript: 안녕하세요 반갑습니다 이게 잘 되는지 테스트 하고 있어요
Transcript:  이거 정말 귀찮은 방법이죠


In [ ]:
client = texttospeech.TextToSpeechClient()

# Performs the list voices request
voices = client.list_voices()

for voice in voices.voices:
    # Display the voice's name. Example: tpc-vocoded
    print(f"Name: {voice.name}")

    # Display the supported language codes for this voice. Example: "en-US"
    for language_code in voice.language_codes:
        print(f"Supported language: {language_code}")

    ssml_gender = texttospeech.SsmlVoiceGender(voice.ssml_gender)

    # Display the SSML Voice Gender
    print(f"SSML Voice Gender: {ssml_gender.name}")

    # Display the natural sample rate hertz for this voice. Example: 24000
    print(f"Natural Sample Rate Hertz: {voice.natural_sample_rate_hertz}\n")


In [ ]:

text = "안녕하세요 반갑습니다"
client = texttospeech.TextToSpeechClient()

input_text = texttospeech.SynthesisInput(text=text)

# Note: the voice can also be specified by name.
# Names of voices can be retrieved with client.list_voices().
voice = texttospeech.VoiceSelectionParams(
    language_code="ko-KR",
    name="ko-KR-Standard-B",
    ssml_gender=texttospeech.SsmlVoiceGender.FEMALE,
)

audio_config = texttospeech.AudioConfig(
    audio_encoding=texttospeech.AudioEncoding.MP3
)

response = client.synthesize_speech(
    request={"input": input_text, "voice": voice, "audio_config": audio_config}
)

# The response's audio_content is binary.
with open("output.mp3", "wb") as out:
    out.write(response.audio_content)
    print('Audio content written to file "output.mp3"')


Audio content written to file "output.mp3"


In [ ]:
Audio("output.mp3")

In [ ]:
kakao_speech_url = "https://kakaoi-newtone-openapi.kakao.com/v1/synthesize"

rest_api_key = '9d0eae41906de495bcb46d11839db95f'

headers = {
    "Content-Type": "application/xml",
    "Authorization": "KakaoAK " + rest_api_key,
}
text = '<speak> 그는 그렇게 말했습니다.<voice name="MAN_DIALOG_BRIGHT">잘 지냈어? 나도 잘 지냈어.</voice> <voice name="WOMAN_DIALOG_BRIGHT" speechStyle="SS_ALT_FAST_1">금요일이 좋아요.</voice> </speak>'

res = requests.post(kakao_speech_url, headers=headers, data=text.encode('utf-8'))
print(res.headers)

with open("output2.mp3", 'wb') as f:
  f.write(res.content)

{'Server': 'openresty', 'Date': 'Sun, 30 May 2021 09:01:54 GMT', 'Content-Type': 'audio/mpeg', 'Transfer-Encoding': 'chunked', 'Connection': 'keep-alive', 'Access-Control-Allow-Origin': '*', 'Vary': 'Origin', 'X-Tts-Length': '41', 'X-Tts-Session-Id': 'ce37836f7b6a5741c14f1c4116f31043451072bc', 'X-Tts-Text': '%EA%B7%B8%EB%8A%94+%EA%B7%B8%EB%A0%87%EA%B2%8C+%EB%A7%90%ED%96%88%EC%8A%B5%EB%8B%88%EB%8B%A4.+%EC%9E%98+%EC%A7%80%EB%83%88%EC%96%B4%3F+%EB%82%98%EB%8F%84+%EC%9E%98+%EC%A7%80%EB%83%88%EC%96%B4.++%EA%B8%88%EC%9A%94%EC%9D%BC%EC%9D%B4+%EC%A2%8B%EC%95%84%EC%9A%94.'}


In [ ]:
Audio("output2.mp3")